In [1]:
# LINHAS: Bronze -> Silver (Parquet, sem Delta)

from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col, explode, current_timestamp, to_date, lit

# --- Paths ---
today = datetime.now().strftime("%Y/%m/%d")
BRONZE_PATH = f"s3a://bronze/linhas/{today}/"   # ajuste se seu prefixo for outro (ex.: linhas_ref)
SILVER_PATH = "s3a://silver/dim_linhas/"

print(f"Lendo Bronze de: {BRONZE_PATH}")
print(f"Gravando Silver em: {SILVER_PATH}")

Lendo Bronze de: s3a://bronze/linhas/2025/11/07/
Gravando Silver em: s3a://silver/dim_linhas/


In [2]:
# --- SparkSession (S3A/MinIO) ---
spark = (
    SparkSession.builder.appName("BronzeToSilver_Linhas_Parquet")
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.access.key", "admin")
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.path.style.access", True)
    .getOrCreate()
)

In [3]:
# --- Schema do JSON de linhas (array na raiz) ---
schema_item = StructType([
    StructField("cl", IntegerType(), True),   # código interno da linha
    StructField("lc", BooleanType(), True),
    StructField("lt", StringType(), True),    # código visível (ex.: "1012")
    StructField("sl", IntegerType(), True),   # sentido
    StructField("tl", IntegerType(), True),   # tipo linha (qdo existir)
    StructField("tp", StringType(), True),    # terminal origem
    StructField("ts", StringType(), True),    # terminal destino
])
schema_array = ArrayType(schema_item)


In [7]:
# --- Leitura robusta (array na raiz) ---
raw = (spark.sparkContext
       .wholeTextFiles(f"{BRONZE_PATH.rstrip('/')}" + "/*.json")
       .toDF(["path","raw"]))
ddl = "array<struct<cl:int, lc:boolean, lt:string, sl:int, tl:int, tp:string, ts:string>>"
df_arr = raw.selectExpr(
    "path",
    f"from_json(raw, '{ddl}') as arr"
    )
df_lin = df_arr.select("path", explode(col("arr")).alias("r"))

# --- Limpeza/seleção (snapshot do dia) ---
df_clean = (
    df_lin
    .select(
        col("r.cl").alias("line_id"),
        col("r.lt").alias("line_code"),
        col("r.sl").alias("sentido"),
        col("r.tl").alias("tipo_linha"),
        col("r.tp").alias("terminal_origem"),
        col("r.ts").alias("terminal_destino"),
    )
    .dropDuplicates(["line_id", "line_code", "sentido"])       # evita duplicatas do mesmo snapshot
    .withColumn("dt", to_date(current_timestamp()))            # partição de snapshot diário
    .withColumn("ingest_ts", current_timestamp())
)

In [8]:
# --- Escrita em Parquet (snapshot diário) ---
# Para snapshot do dia, normalmente usamos overwrite NA PARTIÇÃO do dia.
# Se preferir append puro, troque para .mode("append").
(
    df_clean
    .write
    .mode("overwrite")
    .partitionBy("dt")
    .parquet(SILVER_PATH)
)

print("✅ Linhas: transformação concluída e salva em Parquet na camada Silver.")


✅ Linhas: transformação concluída e salva em Parquet na camada Silver.


In [9]:
# --- Validação rápida ---
df_result = spark.read.parquet(SILVER_PATH)
df_result.orderBy(col("dt").desc(), col("line_id")).show(10, truncate=False)

+-------+---------+-------+----------+-------------------+----------------+-------------------------+----------+
|line_id|line_code|sentido|tipo_linha|terminal_origem    |terminal_destino|ingest_ts                |dt        |
+-------+---------+-------+----------+-------------------+----------------+-------------------------+----------+
|1      |5119     |1      |10        |LGO. SÃO FRANCISCO |TERM. CAPELINHA |2025-11-07 03:44:49.77775|2025-11-07|
|4      |669A     |1      |10        |TERM. PRINC. ISABEL|TERM. STO. AMARO|2025-11-07 03:44:49.77775|2025-11-07|
|5      |6801     |1      |10        |TERM. JOÃO DIAS    |JD. IBIRAPUERA  |2025-11-07 03:44:49.77775|2025-11-07|
|6      |6837     |1      |10        |TERM. CAPELINHA    |SHOP. PORTAL    |2025-11-07 03:44:49.77775|2025-11-07|
|8      |695V     |1      |10        |METRÔ ANA ROSA     |TERM. CAPELINHA |2025-11-07 03:44:49.77775|2025-11-07|
|12     |6835     |1      |10        |TERM. CAPELINHA    |VALO VELHO      |2025-11-07 03:44:49.7